In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.retrievers import TFIDFRetriever
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

In [3]:
embedding = OllamaEmbeddings(model="nomic-embed-text:latest")  # 768 dims

In [4]:
#TF-IDF
tfidf_retriever = TFIDFRetriever.load_local("tfidf_aoe2.pkl", allow_dangerous_deserialization=True)

#Chroma
chroma_vectorstore = Chroma(embedding_function=embedding, persist_directory="chroma_aoe2")
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 5})

#Faiss
faiss_retriever = FAISS.load_local("faiss_aoe2", embedding, allow_dangerous_deserialization=True).as_retriever()

In [5]:
from langchain_core.retrievers import BaseRetriever
from langchain_ollama import ChatOllama
llm_client = ChatOllama(model="llama3.1:latest")

We are using this basic agent described below, no re-ranking is used so far.

In [6]:
class BasicAgent:
    def __init__(self, llm:ChatOllama, retriever:BaseRetriever) -> None:
        self.retriever = retriever
        self.llm = llm

    def custom_retriever(self, user_query:str, k):
        try:
            self.retriever.k = k
        except Exception as e:
            self.retriever.search_kwargs['k'] = k
        retrieved_docs = self.retriever.invoke(user_query)
        context = ""
        for doc in retrieved_docs:
            context += f"Extracted from page {doc.metadata['page']} \n{doc.page_content} \n\n"
        return context, retrieved_docs

    def query(self, user_query:str, k=5):
        context, retrieved_docs = self.custom_retriever(user_query, k)
        system_prompt = ("You are helpful assistant, your role is to assist people getting their way around the rules and mechanics of the famous game Age of Empires 2."
                        "You have the task to answer using the following context"
                        f"<CONTEXT>{context}</CONTEXT>"
                        "Keep you answers brief, make reference to the pagees used and keep the answer at 50 words at max."
                        "If the answer is not contained in the context, say you don't know")
        prompt = self.make_llama_3_prompt(user_query, system_prompt)
        answer = self.llm.invoke(prompt)
        return answer.content, context, retrieved_docs
    
    def make_llama_3_prompt(self, user, system="", context=""):
        if system != "":
            system_prompt = (
                f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
            )
        return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [7]:
TFIDF_Agent = BasicAgent(llm_client, tfidf_retriever)
Chroma_Agent = BasicAgent(llm_client, chroma_retriever)
Faiss_Agent = BasicAgent(llm_client, faiss_retriever)

### Evaluator 
Part of the code was extracted from Huggin Face https://huggingface.co/learn/cookbook/en/rag_evaluation

In [9]:
import json
with open("eval.json", 'r') as f:
    eval_dataset = json.load(f)

In [10]:
eval_dataset[:5]

[{'page_content': 'The heavy camel was an especially experienced warrior and camel rider who wore some armor . They were\nused by desert civilizations of the Middle East who fought against archers from the Byzantine Empire and\nhorse archers raiding down from the steppes of Asia.',
  'metadata': {'page': 78, 'source': 'docs/Age_of_Empires_2_Manual.pdf'},
  'evalqa': {'question': 'What type of camel was known to be an especially experienced warrior and camel rider?',
   'answer': 'The heavy camel was an especially experienced warrior and camel rider.'}},
 {'page_content': 'the game. You can display the objectives again during a game by clicking the Objectives\nbutton at the top of the screen.\nStandard victory\nYou can win any Random Map or Death Match game by being the first player or team todefeat your enemies in military conquest, control all relics, or build a Wonder. You andyour opponents do not have to pursue the same victory condition. For example, you may',
  'metadata': {'page'

In [17]:
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStore
from tqdm.notebook import tqdm

def run_rag_tests(
    agent:BasicAgent, 
    eval_dataset,
    output_file: str,
    verbose = True):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    outputs = []

    for example in tqdm(eval_dataset):
        question = example['evalqa']["question"]

        answer, context, relevant_docs = agent.query(question)
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example['evalqa']["answer"]}')
        result = {
            "question": question,
            "true_answer": example['evalqa']["answer"],
            "source_doc": example["page_content"],
            "generated_answer": answer,
            "retrieved_docs": [doc.page_content for doc in relevant_docs],
            "metadata": [doc.metadata for doc in relevant_docs],
        }

        outputs.append(result)

    with open(output_file, "w") as f:
        json.dump(outputs, f)

In [ ]:
# !pip install --upgrade --quiet jupyter_client ipywidgets

In [19]:
run_rag_tests(TFIDF_Agent, eval_dataset, "tfidf_test.json", verbose=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Question: What type of camel was known to be an especially experienced warrior and camel rider?
Answer: The Heavy Camel (page 78).
True answer: The heavy camel was an especially experienced warrior and camel rider.
Question: What are the standard ways to win a Random Map or Death Match game in this game?
Answer: According to page 18, in a Random Map or Death Match game, you can win by being the first player or team to defeat your enemies in military conquest, control all relics, or build a Wonder.
True answer: You can win any Random Map or Death Match game by being the first player or team to defeat your enemies in military conquest, control all relics, or build a Wonder. Your opponents do not have to pursue the same victory condition.
Question: What types of units can Non-upgraded Monks convert if they research technology at the Monastery?
Answer: Non-upgraded Monks can convert enemy buildings, all siege units, and enemy Monks. (Page 41)
True answer: Non-upgraded Monks can convert ene

In [20]:
run_rag_tests(Chroma_Agent, eval_dataset, "chroma_test.json", verbose=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Question: What type of camel was known to be an especially experienced warrior and camel rider?
Answer: A heavy camel. (Extracted from page 78)
True answer: The heavy camel was an especially experienced warrior and camel rider.
Question: What are the standard ways to win a Random Map or Death Match game in this game?
Answer: To win a Random Map or Death Match game, you can achieve any of the following:

* Defeat your enemies in military conquest (pg 18)
* Control all relics (pg 18)
* Build a Wonder (pg 18)

You don't have to pursue the same victory condition as your opponents.
True answer: You can win any Random Map or Death Match game by being the first player or team to defeat your enemies in military conquest, control all relics, or build a Wonder. Your opponents do not have to pursue the same victory condition.
Question: What types of units can Non-upgraded Monks convert if they research technology at the Monastery?
Answer: Non-upgraded Monks can convert most buildings, all siege u

In [22]:
run_rag_tests(Faiss_Agent, eval_dataset, "faiss_test.json", verbose=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Question: What type of camel was known to be an especially experienced warrior and camel rider?
Answer: According to Context from page 78, the "Heavy Camel" was an especially experienced warrior and camel rider.
True answer: The heavy camel was an especially experienced warrior and camel rider.
Question: What are the standard ways to win a Random Map or Death Match game in this game?
Answer: To win a Random Map or Death Match game in Age of Empires 2, you can achieve one of three standard victory conditions: 

1. Military conquest (no page reference provided) 
2. Control all relics (page 16)
3. Build a Wonder (page 16)

(I could not find any other option on the provided context pages.)
True answer: You can win any Random Map or Death Match game by being the first player or team to defeat your enemies in military conquest, control all relics, or build a Wonder. Your opponents do not have to pursue the same victory condition.
Question: What types of units can Non-upgraded Monks convert i

Now that we have all the answers generate we can go ahead and compute some metrics

---


In [30]:
from langchain_core.documents import Document
documents = chroma_vectorstore.get()
documents = [Document(page_content=page_content, metadata=metadata) for page_content, metadata in zip(documents['documents'], documents['metadatas'])]

In [31]:
documents

[Document(metadata={'page': 96, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='even faster than Double-Bit Axe.\nThe bow saw had a rounded handle like a bow with the saw blade connecting the bow ends. The bow saw was\na more precise tool than previous saws. W oodcutters using it got more usable wood from each tree by reducing\nwaste.\nTwo-Man Saw\nT wo-Man Saw (at the Lumber Camp) makes villagers chop\nwood even faster than the Bow Saw.'),
 Document(metadata={'page': 62, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='60 Chapter VII  -  Units\nMan-at-Arms\nStronger than Militia; cheap and quick to create.\n/c67/c114/c101/c97/c116/c101/c100/c32/c97/c116 Barracks\n/c83/c116/c114/c111/c110/c103/c32/c118/c115 /c46 skirmishers, camels, Light Cavalry\n/c87 /c101/c97/c107/c32/c118/c115 /c46 archers, scorpions, cavalry archers, mangonels, Cataphracts\n/c85/c112/c103/c114/c97/c100/c101/c115 Attack — Forging, Iron Casting, Blast Furnace (Blacksmith)'),
 Document(metada